# 1.基本思想  
基本思想，将所有的bbox按conf的高低排序，将最高conf的bbox提取出来，比较其他框与它的iou，如果类别与之一致且iou大于一定thres，则把该bbox去掉，然后再重复上述步骤  
  
**核心：每个循环提取的也只有conf最高的那个bbox而已**

## 1.1 list数据结构版本  
list中，只能对list中单个元素进行操作，不能像tensor那样一次性切所有的行中的某几个元素为新list这样的操作，所以思想比较好想，只要对每个元素进行iou比较即可，每次提取的同样只有最高的conf那个框而已。

In [1]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])

def iou(box1, box2):
    # 传入的都是两角点四个坐标
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    insert_area = (x2 - x1) * (y2 - y1)
    unioun_area = area(box1) + area(box2)
    return insert_area / (unioun_area - insert_area)

def nms_list(boxes, iou1, iou2):
    boxes.sort(key=lambda x: x[1], reverse=True)
    select_box = []
    while boxes:
        top_box = boxes[0]
        select_box.append(top_box)
        del boxes[0]
        select = []
        for box in boxes:
            if iou(box[2:], top_box[2:]) >= iou2 or (box[0] == top_box[0] and iou(box[2:], top_box[2:]) >= iou1):
                continue
            else:
                select.append(box)
        boxes = select

    return select_box


验证一下！

In [2]:
    
boxes_list = [[0, 0.6, 1, 1, 10, 10], [0, 0.5, 1, 2, 11, 11], [0, 0.6, 1, 1.2, 5, 5], [1, 0.6, 1, 1, 5, 5],[1, 0.9, 1, 1, 5, 5]]
result1 = nms_list(boxes_list, 0.5, 0.7)
print('list', result1)
    
# list [[1, 0.9, 1, 1, 5, 5], [0, 0.6, 1, 1, 10, 10]]

list [[1, 0.9, 1, 1, 5, 5], [0, 0.6, 1, 1, 10, 10]]


## 1.2 tensor版本  
tensor由于不支持对其内部元素删除增加等操作，所以只能通过构造bool进行“切片”，完成删除操作。

In [3]:
import torch

def area_tensor(box):
    if box.dim() > 1:
        return (box[:, 2] - box[:, 0]) * (box[:, 3] - box[:, 1])
    else:
        return (box[2] - box[0]) * (box[3] - box[1])


def iou_tensor(box1, box2):
    x1 = torch.max(box1[:, 0], box2[0])
    y1 = torch.max(box1[:, 1], box2[1])
    x2 = torch.min(box1[:, 2], box2[2])
    y2 = torch.min(box1[:, 3], box2[3])

    intersection_area = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)

    union_area = area_tensor(box1) + area_tensor(box2) - intersection_area
    return intersection_area / union_area


def nms_tensor(boxes, iou1, iou2):
    sorted_value, sorted_indices = torch.sort(boxes[:, 1], descending=True)
    sorted_boxes = boxes[sorted_indices]
    selected_boxes = []

    while sorted_boxes.size(0) > 0:
        top_box = sorted_boxes[0]
        selected_boxes.append(top_box)
        mask = (iou_tensor(sorted_boxes[:, 2:], top_box[2:]) < iou2) & ((sorted_boxes[:, 0] == top_box[0]) | (iou_tensor(sorted_boxes[:, 2:], top_box[2:]) < iou1))
        # mask为： tensor([False, True, False, False, True])
        sorted_boxes = sorted_boxes[mask]
    result_boxes = torch.stack(selected_boxes)
    return result_boxes  # 将list中的两个tensor叠在一块


验证一下！

In [4]:
boxes_tensor = torch.tensor(([0, 0.6, 1, 1, 10, 10], [0, 0.5, 1, 2, 11, 11], [0, 0.6, 1, 1.2, 5, 5],[1, 0.6, 1, 1, 5, 5], [1, 0.9, 1, 1, 5, 5]))
result2 = nms_tensor(boxes_tensor, 0.5, 0.7)
print('tensor', result2)
    
# tensor tensor([[ 1.0000,  0.9000,  1.0000,  1.0000,  5.0000,  5.0000],[ 0.0000,  0.6000,  1.0000,  1.0000, 10.0000, 10.0000]]) 

tensor tensor([[ 1.0000,  0.9000,  1.0000,  1.0000,  5.0000,  5.0000],
        [ 0.0000,  0.6000,  1.0000,  1.0000, 10.0000, 10.0000]])


## 1.3 补充  
tensor里面的mask

In [ ]:
mask = (iou_tensor(sorted_boxes[:, 2:], top_box[2:]) < iou2) \
        & ((sorted_boxes[:, 0] == top_box[0]) | (iou_tensor(sorted_boxes[:, 2:], top_box[2:]) < iou1))

1. and用&表示，or用 | 表示，在这里要将小于阈值的保留框抽出去，所以要反着条件写。
2. 用 !=, >, <等符号都可以进行bool张量的构建，构造完后再进行切片选择操作---tensor没法删除
3. 注意tenosr中的排序与数组中排序的不同，详情见排序问题。